<h1 style="border: thick double #32a1ce; text-align:center;border-radius:35px">
Projet Python pour la data science
</h1>

In [2]:
# Librairies standards
import zipfile
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from math import radians, cos, sin, asin, sqrt
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

<h1 style=" text-align:center">
1.Description des données 
</h1>

<h2 style=" text-align:left">
1.1- Paramètres 
</h2>

In [3]:
# Définition du répertoire de travail
HOME_DIR = Path.cwd().parent
DATA_DIR = Path(HOME_DIR, "data")
print(f"Work directory: {HOME_DIR} \nData directory: {DATA_DIR}")

Work directory: /home/onyxia/work/projet-Python/projet 
Data directory: /home/onyxia/work/projet-Python/projet/data


<h2 style=" text-align:left">
1.2- Chargement des bases
</h2>

Dans le cadre de notre travail nous allons utiliser 05 base bases de données qui porte essentiellement sur: 

<ul>
    <li> la qualité de l'air : indices_QA_commune_IDF_2017.csv</li>
    <li> les maladies : effectifs (4).csv </li>
    
</ul>

   Dans cette partie il est question de chargé et visualiser nos  differentes  bases de données.

<h3 style=" text-align:left">
a. Données sur la qualité de l'air  
</h3>

In [4]:
qualite_air = pd.read_csv(Path(DATA_DIR, "indices_QA_commune_IDF_2017.csv"), sep=",")
qualite_air.head(5)

,date,ninsee,no2,o3,pm10
0,01/01/2017,0,23,16,43
1,01/01/2017,75,23,8,38
2,01/01/2017,77,19,17,44
3,01/01/2017,78,16,15,32
4,01/01/2017,91,14,16,34


In [5]:
qualite_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476485 entries, 0 to 476484
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    476485 non-null  object
 1   ninsee  476485 non-null  int64 
 2   no2     476485 non-null  int64 
 3   o3      476485 non-null  int64 
 4   pm10    476485 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 18.2+ MB


Description : Notre  base ne contient aucune valeur manquante. 

Variable : 
<ul>
    <li>date </li>
    <li>ninsee: code postale des département de l'ile de France</li>
    <li>no2: quantité de NO2 présente dans l'air à cette date</li>
    <li> o3: quantité de 03 présente dans l'air à cette date</li>
    <li> pm10: quantité de PM10 présente dans l'air à cette date</li>
</ul>



In [6]:
quali_air=qualite_air.copy()
var=['no2', 'o3', 'pm10']
quali_air['dept']= qualite_air['ninsee'].astype(str).str[:2].astype(int)


quali_air=quali_air.groupby('dept')[var].mean().reset_index()
vars=['dept','no2', 'o3', 'pm10']
quali_air=round(quali_air[vars],2)
quali_air.head()

,dept,no2,o3,pm10
0,0,37.89,35.77,36.33
1,75,32.50,28.17,33.52
2,77,15.03,31.92,26.75
3,78,17.17,31.91,24.82
4,91,17.32,32.16,25.00


<h3 style=" text-align:left">
b. Données sur les pathologies
</h3>

In [7]:
# Chemins
zip_path = Path(DATA_DIR, "effectifs (4).csv.zip")  # Fichier ZIP
extract_path = Path(DATA_DIR)  # Dossier où extraire les fichiers

# Nom du fichier à extraire depuis l'archive
file_to_extract = "effectifs (4).csv"

# Extraction
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extraire uniquement un fichier spécifique
    effectifs = zip_ref.extract(file_to_extract, extract_path)
    print(f"Fichier {file_to_extract} extrait avec succès dans {extract_path} !")

Fichier effectifs (4).csv extrait avec succès dans /home/onyxia/work/projet-Python/projet/data !


In [13]:
import pandas as pd
pathologie = pd.read_csv(Path(DATA_DIR, "effectifs (4).csv"),sep=';')

In [15]:
pathologie.columns
#on verifie les codes des regions 
print('voici les codes des regions disponible:',pathologie['region'].unique())
#on verifie le type 
print('le type est de:',pathologie['region'].dtypes)

voici les codes des regions disponible: [ 1  2  4  6 11 24 28 32 44 52 53 75 76 84 93 94  3 27 99]
le type est de: int64


In [16]:
#on filtre les données par region (on s'interessent a la region parisienne) .
filtred_by_rg=pathologie[pathologie['region']==11]
#on peut maintenant supprimer la colonne region
filtred_by_rg=filtred_by_rg.drop(columns = ['region'])

In [17]:
#on veut garder que les maladies respiratoirs qui sont affecter par la qualiter de l'aire
new_pathologie = filtred_by_rg[filtred_by_rg['patho_niv2'].isin(['Maladies respiratoires chroniques (hors mucoviscidose)',
        'Cancer bronchopulmonaire'])]

new_pathologie.head(2)

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,dept,Ntop,Npop,prev,Niveau prioritaire,libelle_classe_age,libelle_sexe,tri
13356,2017,Maladies respiratoires chroniques (hors mucovi...,Maladies respiratoires chroniques (hors mucovi...,Maladies respiratoires chroniques (hors mucovi...,RES_CAT_EXC,85-89,2,77,1550.0,11990,12.957,"1,2,3",de 85 à 89 ans,femmes,10.0
13357,2017,Maladies respiratoires chroniques (hors mucovi...,Maladies respiratoires chroniques (hors mucovi...,Maladies respiratoires chroniques (hors mucovi...,RES_CAT_EXC,85-89,2,91,1380.0,11890,11.572,"1,2,3",de 85 à 89 ans,femmes,10.0


In [20]:
print(new_pathologie['Niveau prioritaire'].unique())
print(new_pathologie['tri'].unique())
print(new_pathologie['cla_age_5'].unique())
print(new_pathologie['patho_niv2'].unique())

['1,2,3' '2' '3']
[10. 19. 58. 59.]
['85-89' '90-94' '95et+' 'tsage' '00-04' '05-09' '10-14' '15-19' '20-24'
 '25-29' '30-34' '35-39' '40-44' '45-49' '50-54' '55-59' '60-64' '65-69'
 '70-74' '75-79' '80-84']
['Maladies respiratoires chroniques (hors mucoviscidose)'
 'Cancer bronchopulmonaire']


In [23]:
priority_caount = new_pathologie['Niveau prioritaire'].value_counts()

print(priority_caount)

Niveau prioritaire
3        1134
1,2,3     567
2         567
Name: count, dtype: int64


In [24]:

print("nombre d'observation",new_pathologie.shape[0])



nombre d'observation 2268


In [25]:
#comme que la varible tri n'est pas specifie dans la doccumentation on va exploiter sa correlation avec les 
#autre variables 
#corrélation entre tri et d'autre variable 
corr= new_pathologie.corr(numeric_only=True)['tri']
print(corr)

annee             NaN
sexe     2.727714e-16
Ntop    -1.453304e-01
Npop    -5.261311e-17
prev    -5.846876e-01
tri      1.000000e+00
Name: tri, dtype: float64


In [26]:
#on remarque une forte correlation entre tri et niveau prioritaire 
#on compare donc la distrubition 
print(new_pathologie[['tri', 'Niveau prioritaire']].drop_duplicates())

         tri Niveau prioritaire
13356   10.0              1,2,3
245627  19.0                  2
251462  58.0                  3
257301  59.0                  3


In [27]:
#niveau prioritaire est suffisant pour notre analyse 
#on supprime donc la colonne tri
new_pathologie=new_pathologie.drop(columns =['tri'])

In [28]:
#on adapte une data-set dans les libelles
#on supprime prev pour le calucler plutart 
new_pathologie_1=new_pathologie.drop(columns=['libelle_classe_age','libelle_sexe'])

In [29]:
#new_pathologie_1.head()
print("nombre d'observation",new_pathologie_1.shape[0])

nombre d'observation 2268


<h2 style=" text-align:left">
1.3- Fusion des bases de données 
</h2>